# Object Tracking

In [83]:
import cv2
import numpy as np
import math
import glob
import os
import copy
from tqdm import tqdm
from matplotlib import pyplot as plt

from utils import *

# Lucas Kanade Tracker Implementation

In [2]:
def getDeltaP(b, gradImage, templatePts):
    
    jacobian = getJacobian(templatePts)
    gradImage = np.expand_dims(gradImage, axis=1)

    A = np.matmul(gradImage, jacobian)
    A = np.squeeze(A, axis=1)

    Hessian = np.matmul(A.T, A)
    AT_b = np.matmul(A.T, b)

    deltaP = np.matmul(np.linalg.pinv(Hessian), AT_b)

    return deltaP

In [3]:
def lucasKanadeTracker(templatePts, templateIntensity, Image, p):
    
    threshold = 0.035
    maxIter = 150
    deltaP_norm = np.inf

    gradientX = cv2.Sobel(Image,cv2.CV_64F,1,0,ksize=3)
    gradientY = cv2.Sobel(Image,cv2.CV_64F,0,1,ksize=3)

    for i in range(maxIter):

        W = getWarpMat(p)
        warpedImgPts, warpedImgIntensity, gradXValues, gradYValues = warpPoints(Image, W, templatePts, gradientX, gradientY)

        error = templateIntensity - warpedImgIntensity
        
        gradImage = np.stack((gradXValues, gradYValues), axis=1)
        
        deltaP = getDeltaP(error, gradImage, templatePts)
        deltaP_norm = np.linalg.norm(deltaP)

        p += deltaP

        if (deltaP_norm < threshold):
            break

    return W,p

In [4]:
def trackObject(frames, ROI):
    
    tracker_output = []
    roi_Pt1, roi_Pt2 = np.array([ROI[0], ROI[1], 1]), np.array([ROI[0]+ROI[2], ROI[1]+ROI[3], 1])

    template = frames[...,0]
    templatePts, templateIntensity = getUnrolledPts(template, ROI)
    mean_template = np.mean(template)

    p = np.zeros(6, dtype = float)

    for i in tqdm(range(1, frames.shape[-1])):
        
        frame_current = copy.deepcopy(frames[...,i])
        frame_display = frame_current.copy()
        
        # Normalizing the intensity for new image with respect to template image
        mean_frame = np.mean(frame_current)
        frame_current = frame_current.astype(float)
        frame_current = frame_current *(mean_template/mean_frame)

        # Lucas Kanade Tracker
        warpMat, p = lucasKanadeTracker(templatePts, templateIntensity, frame_current, p)

        roi_Pt1_new = np.matmul(warpMat, roi_Pt1).astype(int)
        roi_Pt2_new = np.matmul(warpMat, roi_Pt2).astype(int)

        cv2.rectangle(frame_display, tuple(roi_Pt1_new), tuple(roi_Pt2_new), (0, 0, 255), 2)
        tracker_output.append(frame_display)

    return tracker_output

# Loading Data

In [5]:
data_dir = '../Data/'
videos = {}

for vid in os.listdir(data_dir):
        if vid.endswith('.npy'):
            videos[vid.split('.')[0]] = np.load(data_dir + vid)

print('Videos Loaded: ', list(videos.keys()))

Videos Loaded:  ['car1', 'car2', 'landing', 'tracker_car1', 'tracker_car2', 'tracker_landing']


In [6]:
play_video(videos['car1'], type='array')

Playing video ....
Press q to exit


# Applying Tracker

## Car_1

In [77]:
selection = 'car1'
frames = videos[selection]

In [8]:
ROI = cv2.selectROI(frames[...,0])
cv2.destroyAllWindows()
# ROI = [118, 103, 226, 179]
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  (127, 107, 205, 166)


In [9]:
tracker_car1 = trackObject(frames, ROI)

100%|██████████| 260/260 [01:19<00:00,  3.28it/s]


In [26]:
play_video(tracker_car1, type='list')

Playing video ....
Press q to exit


In [78]:
# save numpy array as npy file
save_path = f'{data_dir}tracker_{selection}.npy'
save_as_npy(tracker_car1, save_path)
save_as_gif(tracker_car1, f'{data_dir}tracker_{selection}.gif')

Saved as ../Data/tracker_car1.gif


## Car_2

In [79]:
selection = 'car2'
frames = videos[selection]

In [13]:
ROI = cv2.selectROI(frames[...,0])
cv2.destroyAllWindows()
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  (47, 108, 111, 53)


In [14]:
tracker_car2 = trackObject(frames, ROI)

100%|██████████| 414/414 [00:24<00:00, 16.65it/s]


In [32]:
play_video(tracker_car2, type='list')

Playing video ....
Press q to exit


In [80]:
# save numpy array as npy file
save_path = f'{data_dir}tracker_{selection}.npy'
save_as_npy(tracker_car2, save_path)
save_as_gif(tracker_car2, f'{data_dir}tracker_{selection}.gif')

Saved as ../Data/tracker_car2.gif


## Landing

In [81]:
selection = 'landing'
frames = videos[selection]

In [18]:
ROI = cv2.selectROI(frames[...,0])
cv2.destroyAllWindows()
# ROI = [426, 78, 135, 59]
print('Selected Region (x,y,w,h): ', ROI)

Selected Region (x,y,w,h):  (433, 82, 133, 62)


In [19]:
tracker_landing = trackObject(frames, ROI)

100%|██████████| 49/49 [00:08<00:00,  6.00it/s]


In [20]:
play_video(tracker_landing, type='list')

Playing video ....
Press q to exit


In [82]:
# save numpy array as npy file
save_path = f'{data_dir}tracker_{selection}.npy'
save_as_npy(tracker_landing, save_path)
save_as_gif(tracker_landing, f'{data_dir}tracker_{selection}.gif')

Saved as ../Data/tracker_landing.gif
